This tutorial will guide you through the process of analyzing your textual data - from pre-processing it - applying topic modelling algorithms - evaluate the topic model manually and automatically - analyzing it's few applications through visualization.

In [1]:
import gensim
import os, re
import numpy as np
import pandas as pd
import pyLDAvis.gensim

import nltk
nltk.download('wordnet') # download wordnet to be used in lemmatization
from nltk.stem import WordNetLemmatizer

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation
from gensim.corpora import Dictionary
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

Using TensorFlow backend.


[nltk_data] Downloading package wordnet to /Users/parul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


For this tutorial, we will use the kaggle's [fake news dataset](https://www.kaggle.com/mrisdal/fake-news).

## Pre-process

This is one of the most important step in analyzing the text data. If the preprocessing is not good, the algorithm can't do much since we are feeding it a lot of noise, or in other words, **Garbage In Garbage Out**. So let's first clean our data using the following techniques:

1. Stopword removal
2. Strip punctuation
3. Bigram collocation detection (frequently co-occuring tokens). Ex. our topic model will make more sense when 'New' and 'York' are treated as 'New_York'.
4. Lemmatization (converting word to its dictionary form)

In [2]:
df_fake = pd.read_csv('fake.csv')
df_fake[['title', 'text', 'language']].head()
df_fake = df_fake.loc[(pd.notnull(df_fake.text)) & (df_fake.language == 'english')]

# Convert data to tokenized list as required by gensim
texts = []
for line in df_fake.text:
    lowered = line.lower()
    words = re.findall(r'\w+', lowered, flags = re.UNICODE | re.LOCALE)
    texts.append(words)

# for bigram collocation detection
bigram = gensim.models.Phrases(texts)

def preprocess(texts):
    # remove punctuation
    texts = [strip_punctuation(' '.join(line)).split() for line in texts]
    # remove stopwords
    texts = [remove_stopwords(' '.join(line)).split() for line in texts]
    # collocation detection
    texts = [bigram[line] for line in texts]
    # lemmatization 
    lemmatizer = WordNetLemmatizer()
    texts = [[word for word in lemmatizer.lemmatize(' '.join(line), pos='v').split()] for line in texts]
    
    return texts

# pre-processing
texts = preprocess(texts)

# split into training, holdout and test data
training_texts = texts[:5000]
holdout_texts = texts[5000:7500]
test_texts = texts[7500:]

# create dictionary mappings for training data
dictionary = Dictionary(training_texts)

# create corpus using training data's dictionary mappings
training_corpus = [dictionary.doc2bow(text) for text in training_texts]
holdout_corpus = [dictionary.doc2bow(text) for text in holdout_texts]
test_corpus = [dictionary.doc2bow(text) for text in test_texts]

/Users/parul/.virtualenvs/gensim3/lib/python3.4/site-packages/gensim-2.3.0-py3.4-macosx-10.6-intel.egg/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


## Train

Now let's train our topic model which is just a matter of single line with gensim. 

But wait! a cool functionality has recently been added to gensim which can be used to monitor the training progress of our topic model. It basically plots the training statistics (using the evaluation metrics: Coherence, Perplexity, Topic difference and Convergence), which can be used to know if our model has been sufficiently trained or there's still a scope of optimizing it more.

We will need to start the Visdom server for visualization:

`python -m visdom.server`

Visdom browser can now be accessed at http://localhost:8097.

In [3]:
# define perplexity callback for hold_out and test corpus
pl_holdout = PerplexityMetric(corpus=holdout_corpus, logger="visdom", title="Perplexity (hold_out)")
pl_test = PerplexityMetric(corpus=test_corpus, logger="visdom", title="Perplexity (test)")

# remaining metrics
ch_umass = CoherenceMetric(corpus=training_corpus, coherence="u_mass", logger="visdom", title="Coherence (u_mass)")
ch_cv = CoherenceMetric(corpus=training_corpus, texts=training_texts, coherence="c_v", logger="visdom", title="Coherence (c_v)")
diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", title="Diff (kullback_leibler)")
convergence_jc = ConvergenceMetric(distance="jaccard", logger="visdom", title="Convergence (jaccard)")

callbacks = [pl_holdout, pl_test, ch_umass, ch_cv, diff_kl, convergence_jc]

In [9]:
# training LDA model
lda_model = LdaModel(corpus=training_corpus, id2word=dictionary, num_topics=35, passes=50, chunksize=1500, iterations=200, alpha='auto', callbacks=callbacks)

<img src="visdom_graph.png">

In [10]:
lda_model.show_topics(num_topics=5)  # Showing only the top 5 topics

[(29,
  '0.053*"brain_force" + 0.044*"http_www" + 0.029*"health" + 0.028*"widget" + 0.028*"wellness_infowars" + 0.028*"force_html" + 0.028*"utm_source" + 0.028*"infowarsstore_com" + 0.028*"utm_campaign" + 0.028*"utm_medium"'),
 (14,
  '0.013*"general" + 0.012*"navy_retired" + 0.011*"air_force" + 0.008*"army_retired" + 0.006*"modi" + 0.006*"u_s" + 0.006*"election" + 0.006*"retired_major" + 0.005*"jewish_press" + 0.005*"tape"'),
 (3,
  '0.036*"s" + 0.026*"hillary" + 0.012*"t" + 0.012*"hillary_clinton" + 0.012*"people" + 0.011*"donald_trump" + 0.011*"trump" + 0.009*"going" + 0.009*"election" + 0.009*"think"'),
 (10,
  '0.032*"s" + 0.014*"clinton" + 0.006*"public" + 0.005*"government" + 0.005*"u_s" + 0.004*"state" + 0.004*"campaign" + 0.004*"president" + 0.004*"hillary_clinton" + 0.004*"money"'),
 (6,
  '0.042*"s" + 0.011*"said" + 0.011*"t" + 0.010*"like" + 0.009*"time" + 0.008*"people" + 0.008*"know" + 0.006*"going" + 0.006*"think" + 0.005*"m"')]

We can also log the training statistics or access them after the model is trained, for any other custom use case. You can refer to [this](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Training_visualizations.ipynb) notebook for further doc on how to use the API.

Other topic modelling algorithms available in gensim:

In [11]:
# decompose the original matrix of words to maintain key topics
lsi_model = LsiModel(corpus=training_corpus, num_topics=35, id2word=dictionary)
lsi_model.show_topics(num_topics=5)

[(0,
  '0.751*"s" + 0.190*"people" + 0.156*"trump" + 0.115*"said" + 0.114*"clinton" + 0.113*"t" + 0.102*"like" + 0.092*"u_s" + 0.088*"world" + 0.087*"time"'),
 (1,
  '-0.364*"trump" + 0.286*"arabs" + 0.260*"jewish" + 0.244*"arab" + 0.235*"morris" + -0.212*"clinton" + 0.164*"jews" + 0.151*"palestine" + 0.144*"israel" + -0.131*"hillary_clinton"'),
 (2,
  '-0.325*"people" + -0.243*"god" + 0.238*"clinton" + 0.226*"trump" + 0.200*"arabs" + 0.172*"arab" + 0.167*"morris" + -0.138*"t" + -0.137*"like" + -0.133*"world"'),
 (3,
  '0.560*"trump" + -0.283*"u_s" + -0.223*"syria" + 0.206*"donald_trump" + -0.183*"s" + 0.178*"people" + -0.176*"russia" + 0.154*"arabs" + 0.148*"jewish" + 0.128*"morris"'),
 (4,
  '0.374*"trump" + -0.370*"s" + 0.252*"syria" + 0.247*"people" + 0.210*"war" + 0.188*"u_s" + 0.176*"russia" + -0.159*"lesley_stahl" + 0.146*"world" + 0.129*"government"')]

In [8]:
# a completely unsupervised algorithm which figures out the no. of topics on it's own
hdp_model = HdpModel(corpus=training_corpus, id2word=dictionary)
hdp_model.show_topics(num_topics=5)

[(0,
  '0.020*s + 0.005*people + 0.005*trump + 0.004*said + 0.003*clinton + 0.003*t + 0.003*like + 0.003*time + 0.003*world + 0.003*u_s + 0.002*hillary_clinton + 0.002*government + 0.002*new + 0.002*state + 0.002*election + 0.002*2016 + 0.002*hillary + 0.002*president + 0.002*war + 0.002*russia'),
 (1,
  '0.017*s + 0.005*said + 0.005*trump + 0.004*people + 0.003*clinton + 0.003*t + 0.003*black + 0.002*2016 + 0.002*hillary_clinton + 0.002*u_s + 0.002*russia + 0.002*war + 0.002*time + 0.002*like + 0.002*new + 0.002*state + 0.002*election + 0.002*donald_trump + 0.002*government + 0.002*syria'),
 (2,
  '0.012*s + 0.004*clinton + 0.004*trump + 0.003*said + 0.002*2016 + 0.002*time + 0.002*hillary_clinton + 0.002*people + 0.002*election + 0.002*state + 0.001*like + 0.001*t + 0.001*government + 0.001*goldman_sachs + 0.001*donald_trump + 0.001*america + 0.001*university + 0.001*city + 0.001*president + 0.001*fbi'),
 (3,
  '0.007*s + 0.002*white + 0.002*new + 0.002*world + 0.002*2016 + 0.002*peo

## Evaluation

### Manual:

We would like to know if the correct thing has been learned, does the topics inferred make sense as per our text data.

Thanks to pyLDAvis, we can visualise our topic models in a really handy way and inspect what words the topics consist of or how similar the topics are.

In [16]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, training_corpus, dictionary, sort_topics=False)

/Users/parul/.virtualenvs/gensim3/lib/python3.4/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0       2.608900        1       1  0.048837  0.004901
1       2.411384        1       2  0.074038  0.149909
2       1.389716        1       3 -0.137852 -0.028412
3       6.448123        1       4  0.191240 -0.184633
4       0.911346        1       5 -0.148650  0.031719
5       1.913964        1       6  0.022703  0.010757
6      11.105652        1       7  0.170197 -0.102347
7       3.682321        1       8  0.093954  0.110120
8       3.016771        1       9 -0.000380  0.019022
9       7.748293        1      10  0.195386  0.048868
10     10.715944        1      11  0.157829 -0.023780
11      0.825284        1      12 -0.135176 -0.026948
12      1.192501        1      13 -0.093422  0.050129
13      1.668930        1      14 -0.030024  0.067137
14      0.596093        1      15 -0.206705 -0.025634
15      4.304280        1      16  0.094113 -0.115372
16      0.623993        1      17 -0.215910 -0.022810
17      3.098556        1      18  0.073822  0.103113
18      2.044708        1      19 -0.007981 -0.055384
19      0.821778        1      20 -0.107258  0.012679
20      1.325505        1      21 -0.020827 -0.216114
21      4.305621        1      22  0.109621 -0.190182
22      2.870927        1      23  0.089919  0.100621
23      1.647764        1      24  0.030120 -0.107742
24      1.076397        1      25 -0.092149  0.009437
25      1.910693        1      26  0.027344  0.167890
26      0.890878        1      27 -0.070247  0.096604
27      0.537964        1      28 -0.198948  0.001154
28      0.661087        1      29 -0.064530 -0.173078
29      0.781233        1      30 -0.195121 -0.029332
30      1.878196        1      31 -0.001692  0.027127
31      7.746054        1      32  0.199481  0.020306
32      2.355250        1      33  0.059003  0.125322
33      1.233194        1      34 -0.059988  0.063469
34      3.650701        1      35  0.149256  0.081481, topic_info=      Category          Freq             Term         Total  loglift  logprob
term                                                                         
65684  Default   6913.000000            trump   6913.000000  30.0000  30.0000
26324  Default   2622.000000           russia   2622.000000  29.0000  29.0000
50256  Default  28712.000000                s  28712.000000  28.0000  28.0000
33606  Default   4769.000000          clinton   4769.000000  27.0000  27.0000
67085  Default   2822.000000          hillary   2822.000000  26.0000  26.0000
57418  Default   3047.000000         election   3047.000000  25.0000  25.0000
48633  Default   1891.000000            syria   1891.000000  24.0000  24.0000
38943  Default   1708.000000              fbi   1708.000000  23.0000  23.0000
11175  Default   3367.000000  hillary_clinton   3367.000000  22.0000  22.0000
52098  Default   2553.000000     donald_trump   2553.000000  21.0000  21.0000
71862  Default   3567.000000              u_s   3567.000000  20.0000  20.0000
28717  Default   2653.000000              war   2653.000000  19.0000  19.0000
3374   Default   5452.000000             said   5452.000000  18.0000  18.0000
45359  Default    910.000000     infowars_com    910.000000  17.0000  17.0000
26053  Default   1477.000000           emails   1477.000000  16.0000  16.0000
63069  Default   7477.000000           people   7477.000000  15.0000  15.0000
2306   Default   3286.000000       government   3286.000000  14.0000  14.0000
46657  Default   2090.000000            obama   2090.000000  13.0000  13.0000
53598  Default   3510.000000            world   3510.000000  12.0000  12.0000
49241  Default   3045.000000            state   3045.000000  11.0000  11.0000
23382  Default   1186.000000            women   1186.000000  10.0000  10.0000
72466  Default   1644.000000             vote   1644.000000   9.0000   9.0000
32945  Default   4484.000000                t   4484.000000   8.0000   8.0000
7796   De

### Automatic:

Coherence is often used to get past the manual inspection and objectively compare the topic models. By returning a score, we can compare between different topic models of the same corpus.

In [13]:
# LDA model coherence
CoherenceModel(lda_model, texts=training_texts, dictionary=dictionary, window_size=10).get_coherence()

0.43647287708995514

In [14]:
# LSI model coherence
CoherenceModel(lsi_model, texts=training_texts, dictionary=dictionary, window_size=10).get_coherence()

0.30212940791550469

In [15]:
# HDP model coherence
CoherenceModel(hdp_model, texts=training_texts, dictionary=dictionary, window_size=10).get_coherence()

0.34361949892725863